<a href="https://colab.research.google.com/github/min03027/IT-DA-DeepLearning/blob/main/chapter_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#신경망 학습
학습 : 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것을 뜻합니다. 
손실함수 : 신경망이 학습할 수 있도록 해주는 지표
손실함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는것이 학습의 목표
-> 이번장에서는 함수의 기울기를 활용하는 경사법 소개

#4.1 데이터에서 학습한다!

신경망의 특징? 데이터를 보고 학습할 수 있다는 것
신경망 학습을 통해 데이터로부터 매개변수의 값을 정하는 방법

2장의 퍼셉트론도 선형분리가능한 문제라면 데이터를 자동으로 학습할 수 있다.
퍼셉트론 수렴정리를 통해 증명됨, 하지만 비선형 분리 문제는 학습할 수 없음


#4.1.1 데이터 주도 학습

데이터에서 답을 찾고 데이터에서 패턴을 찾는다. 기계학습의 중심에는 데이터가 존재한다,

기계학습에서는 사람의 개입을 최소화하고 수집한 데이터로부터 패턴을 찾으려 시도한다. 
사람의 개입을 더욱 배제할 수 있다. 

이미지에서 특징을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법
특징은?
입력데이터(입력 이미지)에서 본질적인 데이터(중요한 데이터)를 정확하게 추출할 수 있도록 설계된 변환기

딥러닝 : 종단간 기계학습 (처음부터 끝까지 , 데이터 입력값에서 목표한 결과를 사람의 개입 없이 얻는다는 것

산경망은 모든 문제를 주어진 데이터 그대로를 입력 데이터로 활용해 end-to-end로 학습 할 수 있다.

#4.1.2 훈련 데이터와 시험 데이터
훈련 데이터와 시험 데이터
훈련데이터를 사용하여 최적의 매개변수를 찾는다. 
이후 시험 데이터를 사용하여 앞서 훈련한 모델의 실력을 평가하는 것
나누는 이유?
범용적으로 사용할 수 있기 떄문에, 범용능력을 제대로 평가하기 위해 훈련데이터 와 시험데이터를 분리하는 것

범용능력은 아직 보지 못한 데이터로도 문제를 올바르게 풀어내는 능력이다
'특정인의 특정 글자'가 아니라 '임의의 사람의 임의의 글자'를 잘인식해야한다. 

오버피팅 : 한 데이터셋에만 지나치게 최적화된 상태 

#4.2 손실함수
신경망 학습에서는 현재의 상태를 하나의 지표로 표현합니다. 
그리고 그 지표를 만들어주는 가중치 매개변수의 값을 탐색하는 것 

손실함수는 보통 오차제곱합과 교차 엔트로피 오차를 사용한다.


#4.2.1 오차제곱합

원-핫 인코딩 : 한 원소만 1로 하고 그 외는 0으로 나타내는 표기법

오차제곱합 
: 각 원소의 출력(추정값)과 정답 레이블(참 값)의 차를 제곱한 후, 그 총합을 구한다. 

In [ ]:
import numpy as np

In [ ]:
def sum_squares_error(y,t):
  return 0.5 * np.sum((y-t)**2)


In [ ]:
t =[0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
sum_squares_error(np.array(y),np.array(t))


0.09750000000000003

In [ ]:
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
sum_squares_error(np.array(y),np.array(t))

0.5975

#4.2.2 교차 엔트로피 오차

교차 엔트로피 오차는 정답일 때의 출력이 전체 값을 정하게 된다. 


In [ ]:
def cross_entropy_error(y,t):
  delta = 1e-7 #마이너스가 되면 계산할 수 없어지기 때문에 매우 작은 값을 더해서 절대 0이 되지 않도록 해준다.
  return -np.sum(t*np.log(y+delta)) 

In [ ]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

0.510825457099338

In [ ]:
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
cross_entropy_error(np.array(y),np.array(t))

2.302584092994546

#4.2.3 미니배치 학습

데이러 하나에 대한 손실 함수를 넘어서 
훈련 데이터 모두에 대한 손실함수의 합을 구하는 방법
데이터 하나에 대한 손실함수를 단순히 N개의 데이터로 확정한다. 다만 마지막에 n으로 나누어 정규화한다. 이를 통해 '평균 손실 함수'를 구한다. 

신경망 학습에서 훈련 데이터로 부터 일부만 골라 학습을 수행한다.(근사치)
이 일부를 미니배치라 한다. 또한 이를 이용한 학습을 미니배치 학습이라 한다. 